# OpenCV Camera Calibration Tuning
This notebook helps you tune parameters for `cv2.findCirclesGrid` using `cv2.SimpleBlobDetector`.

1. **Load Image**: Ensure your image path is correct.
2. **Tune Blob Detector**: Adjust thresholds and filters until blobs (circles) are consistently detected (red circles).
3. **Find Grid**: Set the grid pattern size (e.g. 21x17) and check if the grid is found (colored corners).

In [ ]:
import marimo as mo
import cv2
import sys
import subprocess
import matplotlib.pyplot as plt
from skimage import io
import os
import numpy as np

In [ ]:
img_path = "/home/user/Dropbox/3DPTV_Illmenau/00000093_0000000018B72D72.png"
print(f"Path exists: {os.path.exists(img_path)}")

if os.path.exists(img_path):
    img_cv = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
else:
    img_cv = np.zeros((100, 100), dtype=np.uint8)

img_disp = img_cv.copy()

mo.md(f"Loaded image: `{img_path}` shape={img_cv.shape}")

Path exists: True


<span class="markdown prose dark:prose-invert contents"><span class="paragraph">Loaded image: <code>/home/user/Dropbox/3DPTV_Illmenau/00000093_0000000018B72D72.png</code> shape=(2048, 2560)</span></span>

In [ ]:
# Configuration UI
mo.md("## Configuration")

# Blob Detector
min_thresh = mo.ui.slider(0, 255, value=10, label="Min Threshold")
max_thresh = mo.ui.slider(0, 255, value=200, label="Max Threshold")
filter_area = mo.ui.checkbox(value=True, label="Filter by Area")
min_area = mo.ui.number(start=0, stop=10000, value=10, label="Min Area")
max_area = mo.ui.number(start=0, stop=100000, value=5000, label="Max Area")
filter_circ = mo.ui.checkbox(value=False, label="Filter by Circularity")
min_circ = mo.ui.slider(
    0.0, 1.0, value=0.1, step=0.01, label="Min Circularity"
)
filter_conv = mo.ui.checkbox(value=True, label="Filter by Convexity")
min_conv = mo.ui.slider(0.0, 1.0, value=0.87, step=0.01, label="Min Convexity")
filter_inert = mo.ui.checkbox(value=True, label="Filter by Inertia")
min_inert = mo.ui.slider(0.0, 1.0, value=0.01, step=0.01, label="Min Inertia")

# Grid Pattern
grid_rows = mo.ui.number(start=3, stop=50, value=21, label="Rows")
grid_cols = mo.ui.number(start=3, stop=50, value=17, label="Cols")
flags_clustering = mo.ui.checkbox(value=False, label="Use Clustering")

ui = mo.vstack(
    [
        mo.md("### Blob Detector Settings"),
        mo.hstack([min_thresh, max_thresh]),
        mo.hstack([filter_area, min_area, max_area]),
        mo.hstack([filter_circ, min_circ]),
        mo.hstack([filter_conv, min_conv]),
        mo.hstack([filter_inert, min_inert]),
        mo.md("### Grid Pattern Settings"),
        mo.hstack([grid_rows, grid_cols, flags_clustering]),
    ]
)
# ui

In [ ]:
# Processing
# Setup Blob Detector
params = cv2.SimpleBlobDetector_Params()

params.minThreshold = min_thresh.value
params.maxThreshold = max_thresh.value
params.filterByArea = filter_area.value
params.minArea = min_area.value
params.maxArea = max_area.value
params.filterByCircularity = filter_circ.value
params.minCircularity = min_circ.value
params.filterByConvexity = filter_conv.value
params.minConvexity = min_conv.value
params.filterByInertia = filter_inert.value
params.minInertiaRatio = min_inert.value

detector = cv2.SimpleBlobDetector_create(params)
keypoints = detector.detect(img_cv)
print(keypoints)

error: OpenCV(4.13.0) /io/opencv/modules/features2d/src/blobdetector.cpp:87: error: (-5:Bad argument) 0<=minThreshold<=maxThreshold in function 'validateParameters'

In [ ]:
# Draw detected blobs
im_with_keypoints = cv2.drawKeypoints(
    img_cv,
    keypoints,
    np.array([]),
    (0, 0, 255),
    cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
)

Ancestor raised: An ancestor raised an exception (error)

In [ ]:
# Find Circles Grid
pattern_size = (grid_cols.value, grid_rows.value)
flags = cv2.CALIB_CB_SYMMETRIC_GRID
if flags_clustering.value:
    flags |= cv2.CALIB_CB_CLUSTERING

found, corners = cv2.findCirclesGrid(
    img_cv, pattern_size, flags=flags, blobDetector=detector
)

# Draw Grid
if found:
    cv2.drawChessboardCorners(im_with_keypoints, pattern_size, corners, found)
    status_msg = f"**Success!** Grid found ({len(corners)} points)."
else:
    status_msg = f"**Grid Not Found.** Detected {len(keypoints)} blobs. Adjust parameters."

Ancestor raised: An ancestor raised an exception (error)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(im_with_keypoints)
plt.title(
    f"Blobs: {len(keypoints)} | Grid: {'Found' if found else 'Not Found'}"
)
plt.axis("off")
plot = plt.gca()

mo.vstack([ui, plot])

Ancestor raised: An ancestor raised an exception (error)